# Getting Bigquery to work in a Jupyter notebook
## Maybe this could be used within any of the compute environments made available to us

In [ ]:
!pip install --upgrade google-cloud-bigquery -q

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Set your Google Cloud Project ID
project_id = 'lcp-consortium'

In [ ]:
# Set your Google Cloud Project ID
project_id = 'dukedatathon2025'

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Create a BigQuery client
client = bigquery.Client(project=project_id)

In [ ]:
query = """
WITH patient_list AS (
  SELECT subject_id
  FROM `physionet-data.mimiciv_3_1_hosp.patients`
),

patient_with_selected_route_antibiotics AS (
  SELECT DISTINCT subject_id
  FROM `physionet-data.mimiciv_derived.antibiotic`
  WHERE UPPER(route) IN ('IV', 'PO', 'PT')
),

antibiotic_status AS (
  SELECT
    pl.subject_id,
    CASE
      WHEN pwsa.subject_id IS NULL THEN 'No Antibiotics'
      ELSE 'Received Antibiotics'
    END AS antibiotic_status
  FROM patient_list pl
  LEFT JOIN patient_with_selected_route_antibiotics pwsa
    ON pl.subject_id = pwsa.subject_id
),

cdiff_tests AS (
  SELECT DISTINCT subject_id,
    CASE
      WHEN test_itemid IN (90212, 90221, 90253) THEN
        CASE
          WHEN LOWER(org_name) LIKE '%clostridium difficile%' THEN 'Positive'
          WHEN org_name IS NULL THEN 'Negative'
          WHEN UPPER(org_name) = 'CANCELLED' THEN 'Not Tested'
          ELSE 'Not Tested'
        END
      WHEN test_itemid = 90233 THEN
        CASE
          WHEN LOWER(org_name) LIKE '%positive%' THEN 'Positive'
          WHEN UPPER(org_name) = 'NEGATIVE' THEN 'Negative'
          ELSE 'Not Tested'
        END
      ELSE 'Not Tested'
    END AS cdiff_result
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE test_itemid IN (90212, 90221, 90233, 90253)
),

patient_summary AS (
  SELECT
    a.subject_id,
    a.antibiotic_status,
    CASE
      WHEN c.cdiff_result IS NULL THEN 'Not Tested'
      ELSE c.cdiff_result
    END AS cdiff_status
  FROM antibiotic_status a
  LEFT JOIN cdiff_tests c
    ON a.subject_id = c.subject_id
)

SELECT
  antibiotic_status,
  cdiff_status,
  COUNT(*) AS num_patients
FROM patient_summary
GROUP BY antibiotic_status, cdiff_status
ORDER BY antibiotic_status, cdiff_status;

        """

query_job = client.query(query)

In [ ]:
df = query_job.to_dataframe()
df

NameError: name 'query_job' is not defined

## PPI query

In [ ]:
# ppi query
ppi_query = """

SELECT emar.*
FROM `physionet-data.mimiciv_3_1_hosp.prescriptions` prescriptions
  INNER JOIN `physionet-data.mimiciv_derived.icustay_detail` icudetail
    ON prescriptions.hadm_id = icudetail.hadm_id
  INNER JOIN `physionet-data.mimiciv_3_1_hosp.emar` emar
    ON prescriptions.pharmacy_id = emar.pharmacy_id
WHERE 1 = 1
  --AND icudetail.first_icu_stay IS TRUE	 -- Commented out due to time constraints
    AND emar.hadm_id iS NOT NULL
    AND (LOWER(prescriptions.drug) LIKE '%pantoprazole%'
        OR LOWER(prescriptions.drug) LIKE '%protonix%'
        OR LOWER(prescriptions.drug) LIKE '%omeprazole%'
        OR LOWER(prescriptions.drug) LIKE '%prilosec%'
        OR LOWER(prescriptions.drug) LIKE '%esomeprazole%'
        OR LOWER(prescriptions.drug) LIKE '%nexium%'
        OR LOWER(prescriptions.drug) LIKE '%rabeprazole%'
        OR LOWER(prescriptions.drug) LIKE '%aciphex%'
        OR LOWER(prescriptions.drug) LIKE '%lansoprazole%'
        OR LOWER(prescriptions.drug) LIKE '%prevacid%'
        OR LOWER(prescriptions.drug) LIKE '%dexlansoprazole%'
        OR LOWER(prescriptions.drug) LIKE '%dexilant%')
  AND prescriptions.starttime BETWEEN icudetail.icu_intime AND icudetail.icu_outtime
  AND event_txt = 'Administered'
ORDER BY emar.hadm_id, emar.charttime

"""

In [ ]:
query_job = client.query(ppi_query)
df_ppi = query_job.to_dataframe()
df_ppi

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,enter_provider_id,charttime,medication,event_txt,scheduletime,storetime
0,16679562,20001395,16679562-370,370,16679562-1139,39145761,None,2180-04-24 09:06:00,Omeprazole,Administered,2180-04-24 08:00:00,2180-04-24 09:06:00
1,16679562,20001395,16679562-393,393,16679562-1183,39145761,None,2180-04-25 08:12:00,Omeprazole,Administered,2180-04-25 08:00:00,2180-04-25 08:13:00
2,16679562,20001395,16679562-414,414,16679562-1183,39145761,None,2180-04-26 08:14:00,Omeprazole,Administered,2180-04-26 08:00:00,2180-04-26 08:15:00
3,16679562,20001395,16679562-439,439,16679562-1183,39145761,None,2180-04-27 09:13:00,Omeprazole,Administered,2180-04-27 08:00:00,2180-04-27 09:14:00
4,16679562,20001395,16679562-464,464,16679562-1183,39145761,None,2180-04-28 08:01:00,Omeprazole,Administered,2180-04-28 08:00:00,2180-04-28 08:08:00
...,...,...,...,...,...,...,...,...,...,...,...,...
168433,16199040,29998463,16199040-896,896,16199040-1152,63430966,P82MLY,2163-03-04 08:20:00,Pantoprazole,Administered,2163-03-04 08:00:00,2163-03-04 08:25:00
168434,16199040,29998463,16199040-918,918,16199040-1152,63430966,P07AAV,2163-03-05 07:53:00,Pantoprazole,Administered,2163-03-05 08:00:00,2163-03-05 07:54:00
168435,16199040,29998463,16199040-937,937,16199040-1152,63430966,P34J9U,2163-03-06 08:44:00,Pantoprazole,Administered,2163-03-06 08:00:00,2163-03-06 08:45:00
168436,16968104,29998702,16968104-75,75,16968104-297,81667832,None,2190-11-16 09:31:00,Omeprazole,Administered,2190-11-16 08:00:00,2190-11-16 09:31:00


In [ ]:
df_ppi.columns

Index(['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'pharmacy_id',
       'enter_provider_id', 'charttime', 'medication', 'event_txt',
       'scheduletime', 'storetime'],
      dtype='object')

## Antibiotics query

In [ ]:
# antibiotics query - include hospital and icu
antibiotics_query = """
SELECT *
FROM `physionet-data.mimiciv_derived.antibiotic`
WHERE (
    UPPER(route) IN ('IV', 'IM', 'PO')
    OR UPPER(route) LIKE '%TUBE%'
    OR UPPER(route) LIKE '%ORAL%'
    OR UPPER(route) LIKE '%INJECT%'
)
AND (
LOWER(antibiotic) LIKE '%adoxa%'
    OR LOWER(antibiotic) LIKE '%ala-tet%'
    OR LOWER(antibiotic) LIKE '%alodox%'
    OR LOWER(antibiotic) LIKE '%amikacin%'
    OR LOWER(antibiotic) LIKE '%amikin%'
    OR LOWER(antibiotic) LIKE '%amoxicill%'
    OR LOWER(antibiotic) LIKE '%ancef%'
    OR LOWER(antibiotic) LIKE '%clavulanate%'
    OR LOWER(antibiotic) LIKE '%ampicillin%'
    OR LOWER(antibiotic) LIKE '%augmentin%'
    OR LOWER(antibiotic) LIKE '%avelox%'
    OR LOWER(antibiotic) LIKE '%avidoxy%'
    OR LOWER(antibiotic) LIKE '%azactam%'
    OR LOWER(antibiotic) LIKE '%azithromycin%'
    OR LOWER(antibiotic) LIKE '%aztreonam%'
    OR LOWER(antibiotic) LIKE '%axetil%'
    OR LOWER(antibiotic) LIKE '%bactocill%'
    OR LOWER(antibiotic) LIKE '%bactrim%'
    OR LOWER(antibiotic) LIKE '%bactroban%'
    OR LOWER(antibiotic) LIKE '%bethkis%'
    OR LOWER(antibiotic) LIKE '%biaxin%'
    OR LOWER(antibiotic) LIKE '%bicillin l-a%'
    OR LOWER(antibiotic) LIKE '%cayston%'
    OR LOWER(antibiotic) LIKE '%cefazolin%'
    OR LOWER(antibiotic) LIKE '%cedax%'
    OR LOWER(antibiotic) LIKE '%cefoxitin%'
    OR LOWER(antibiotic) LIKE '%ceftazidime%'
    OR LOWER(antibiotic) LIKE '%cefaclor%'
    OR LOWER(antibiotic) LIKE '%cefadroxil%'
    OR LOWER(antibiotic) LIKE '%cefdinir%'
    OR LOWER(antibiotic) LIKE '%cefditoren%'
    OR LOWER(antibiotic) LIKE '%cefepime%'
    OR LOWER(antibiotic) LIKE '%cefotan%'
    OR LOWER(antibiotic) LIKE '%cefotetan%'
    OR LOWER(antibiotic) LIKE '%cefotaxime%'
    OR LOWER(antibiotic) LIKE '%ceftaroline%'
    OR LOWER(antibiotic) LIKE '%cefpodoxime%'
    OR LOWER(antibiotic) LIKE '%cefpirome%'
    OR LOWER(antibiotic) LIKE '%cefprozil%'
    OR LOWER(antibiotic) LIKE '%ceftibuten%'
    OR LOWER(antibiotic) LIKE '%ceftin%'
    OR LOWER(antibiotic) LIKE '%ceftriaxone%'
    OR LOWER(antibiotic) LIKE '%cefuroxime%'
    OR LOWER(antibiotic) LIKE '%cephalexin%'
    OR LOWER(antibiotic) LIKE '%cephalothin%'
    OR LOWER(antibiotic) LIKE '%cephapririn%'
    OR LOWER(antibiotic) LIKE '%chloramphenicol%'
    OR LOWER(antibiotic) LIKE '%cipro%'
    OR LOWER(antibiotic) LIKE '%ciprofloxacin%'
    OR LOWER(antibiotic) LIKE '%claforan%'
    OR LOWER(antibiotic) LIKE '%clarithromycin%'
    OR LOWER(antibiotic) LIKE '%cleocin%'
    OR LOWER(antibiotic) LIKE '%clindamycin%'
    OR LOWER(antibiotic) LIKE '%cubicin%'
    OR LOWER(antibiotic) LIKE '%dicloxacillin%'
    OR LOWER(antibiotic) LIKE '%dirithromycin%'
    OR LOWER(antibiotic) LIKE '%doryx%'
    OR LOWER(antibiotic) LIKE '%doxycy%'
    OR LOWER(antibiotic) LIKE '%duricef%'
    OR LOWER(antibiotic) LIKE '%dynacin%'
    OR LOWER(antibiotic) LIKE '%ery-tab%'
    OR LOWER(antibiotic) LIKE '%eryped%'
    OR LOWER(antibiotic) LIKE '%eryc%'
    OR LOWER(antibiotic) LIKE '%erythrocin%'
    OR LOWER(antibiotic) LIKE '%erythromycin%'
    OR LOWER(antibiotic) LIKE '%factive%'
    OR LOWER(antibiotic) LIKE '%flagyl%'
    OR LOWER(antibiotic) LIKE '%fortaz%'
    OR LOWER(antibiotic) LIKE '%furadantin%'
    OR LOWER(antibiotic) LIKE '%garamycin%'
    OR LOWER(antibiotic) LIKE '%gentamicin%'
    OR LOWER(antibiotic) LIKE '%kanamycin%'
    OR LOWER(antibiotic) LIKE '%keflex%'
    OR LOWER(antibiotic) LIKE '%kefzol%'
    OR LOWER(antibiotic) LIKE '%ketek%'
    OR LOWER(antibiotic) LIKE '%levaquin%'
    OR LOWER(antibiotic) LIKE '%levofloxacin%'
    OR LOWER(antibiotic) LIKE '%lincocin%'
    OR LOWER(antibiotic) LIKE '%linezolid%'
    OR LOWER(antibiotic) LIKE '%macrobid%'
    OR LOWER(antibiotic) LIKE '%macrodantin%'
    OR LOWER(antibiotic) LIKE '%maxipime%'
    OR LOWER(antibiotic) LIKE '%mefoxin%'
    OR LOWER(antibiotic) LIKE '%metronidazole%'
    OR LOWER(antibiotic) LIKE '%meropenem%'
    OR LOWER(antibiotic) LIKE '%methicillin%'
    OR LOWER(antibiotic) LIKE '%minocin%'
    OR LOWER(antibiotic) LIKE '%minocycline%'
    OR LOWER(antibiotic) LIKE '%monodox%'
    OR LOWER(antibiotic) LIKE '%monurol%'
    OR LOWER(antibiotic) LIKE '%morgidox%'
    OR LOWER(antibiotic) LIKE '%moxatag%'
    OR LOWER(antibiotic) LIKE '%moxifloxacin%'
    OR LOWER(antibiotic) LIKE '%mupirocin%'
    OR LOWER(antibiotic) LIKE '%myrac%'
    OR LOWER(antibiotic) LIKE '%nafcillin%'
    OR LOWER(antibiotic) LIKE '%neomycin%'
    OR LOWER(antibiotic) LIKE '%nicazel doxy 30%'
    OR LOWER(antibiotic) LIKE '%nitrofurantoin%'
    OR LOWER(antibiotic) LIKE '%norfloxacin%'
    OR LOWER(antibiotic) LIKE '%noroxin%'
    OR LOWER(antibiotic) LIKE '%ocudox%'
    OR LOWER(antibiotic) LIKE '%ofloxacin%'
    OR LOWER(antibiotic) LIKE '%omnicef%'
    OR LOWER(antibiotic) LIKE '%oracea%'
    OR LOWER(antibiotic) LIKE '%oraxyl%'
    OR LOWER(antibiotic) LIKE '%oxacillin%'
    OR LOWER(antibiotic) LIKE '%pc pen vk%'
    OR LOWER(antibiotic) LIKE '%pce dispertab%'
    OR LOWER(antibiotic) LIKE '%panixine%'
    OR LOWER(antibiotic) LIKE '%penicillin%'
    OR LOWER(antibiotic) LIKE '%periostat%'
    OR LOWER(antibiotic) LIKE '%pfizerpen%'
    OR LOWER(antibiotic) LIKE '%piperacillin%'
    OR LOWER(antibiotic) LIKE '%tazobactam%'
    OR LOWER(antibiotic) LIKE '%primsol%'
    OR LOWER(antibiotic) LIKE '%proquin%'
    OR LOWER(antibiotic) LIKE '%raniclor%'
    OR LOWER(antibiotic) LIKE '%rifadin%'
    OR LOWER(antibiotic) LIKE '%rifampin%'
    OR LOWER(antibiotic) LIKE '%rocephin%'
    OR LOWER(antibiotic) LIKE '%smz-tmp%'
    OR LOWER(antibiotic) LIKE '%smx-tmp%'
    OR LOWER(antibiotic) LIKE '%septra%'
    OR LOWER(antibiotic) LIKE '%septra ds%'
    OR LOWER(antibiotic) LIKE '%septra%'
    OR LOWER(antibiotic) LIKE '%solodyn%'
    OR LOWER(antibiotic) LIKE '%spectracef%'
    OR LOWER(antibiotic) LIKE '%streptomycin%'
    OR LOWER(antibiotic) LIKE '%sulfadiazine%'
    OR LOWER(antibiotic) LIKE '%sulfamethoxazole%'
    OR LOWER(antibiotic) LIKE '%trimethoprim%'
    OR LOWER(antibiotic) LIKE '%sulfatrim%'
    OR LOWER(antibiotic) LIKE '%sulfisoxazole%'
    OR LOWER(antibiotic) LIKE '%suprax%'
    OR LOWER(antibiotic) LIKE '%synercid%'
    OR LOWER(antibiotic) LIKE '%tazicef%'
    OR LOWER(antibiotic) LIKE '%tetracycline%'
    OR LOWER(antibiotic) LIKE '%timentin%'
    OR LOWER(antibiotic) LIKE '%tobramycin%'
    OR LOWER(antibiotic) LIKE '%trimethoprim%'
    OR LOWER(antibiotic) LIKE '%unasyn%'
    OR LOWER(antibiotic) LIKE '%vancocin%'
    OR LOWER(antibiotic) LIKE '%vancomycin%'
    OR LOWER(antibiotic) LIKE '%vantin%'
    OR LOWER(antibiotic) LIKE '%vibativ%'
    OR LOWER(antibiotic) LIKE '%vibra-tabs%'
    OR LOWER(antibiotic) LIKE '%vibramycin%'
    OR LOWER(antibiotic) LIKE '%zinacef%'
    OR LOWER(antibiotic) LIKE '%zithromax%'
    OR LOWER(antibiotic) LIKE '%zosyn%'
    OR LOWER(antibiotic) LIKE '%zyvox%'
)
"""
query_job = client.query(antibiotics_query)

In [ ]:
df_abx = query_job.to_dataframe()
print(f'''
Number of unique patients: {df_abx['subject_id'].nunique()}
Number of unique hadm_id: {df_abx['hadm_id'].nunique()}
Number of unique antibiotics: {df_abx['antibiotic'].nunique()}
''')


Number of unique patients: 114760
Number of unique hadm_id: 207078
Number of unique antibiotics: 198



In [ ]:
df_abx.head()

,subject_id,hadm_id,stay_id,antibiotic,route,starttime,stoptime
0,13637586,22000149,<NA>,*NF* Cefoxitin Sodium,IV,2156-10-14 17:00:00,2156-10-15 21:00:00
1,18132540,20668522,<NA>,*NF* Ceftaroline,IV,2172-01-03 22:00:00,2172-01-10 16:00:00
2,11249454,24303597,<NA>,*NF* Ceftaroline,IV,2164-11-28 20:00:00,2164-11-30 21:00:00
3,18132540,20668522,<NA>,*NF* Ceftaroline,IV,2172-01-03 22:00:00,2172-01-03 21:00:00
4,18132540,25341960,<NA>,*NF* Ceftaroline,IV,2171-12-09 18:00:00,2171-12-11 10:00:00


In [ ]:
df_abx.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'antibiotic', 'route', 'starttime',
       'stoptime'],
      dtype='object')

In [ ]:
# prompt: I want to count the number of rows where the route is PO and the antibiotic includes the text *vanco*

# Count rows where route is 'PO' and antibiotic contains 'vanco'
vanco_po_count = df_abx[(df_abx['route'] == 'PO') & (df_abx['antibiotic'].str.lower().str.contains('vanco'))].shape[0]

print(f"Number of rows with route 'PO' and antibiotic containing 'vanco': {vanco_po_count}")


Number of rows with route 'PO' and antibiotic containing 'vanco': 1548


## C diff query

In [ ]:
# cdiff query - include hospital and icu
cdiff_query = """
  SELECT hosp.subject_id, hosp.hadm_id, hosp.chartdate, hosp.charttime,
       hosp.test_itemid, hosp.test_name, hosp.org_name, hosp.comments,
       CASE
           WHEN hosp.test_itemid IN (90212, 90221, 90253) THEN
               CASE
                   WHEN LOWER(hosp.org_name) LIKE '%clostridium difficile%' THEN 'Positive'
                   WHEN hosp.org_name IS NULL THEN 'Negative'
                   WHEN LOWER(hosp.comments) LIKE '%negative%' THEN 'Negative'
                   WHEN UPPER(hosp.org_name) = 'CANCELLED' THEN 'Not Tested'
                   ELSE 'Not Tested'
               END
           WHEN hosp.test_itemid = 90233 THEN
               CASE
                   WHEN LOWER(hosp.org_name) LIKE '%positive%' THEN 'Positive'
                   WHEN UPPER(hosp.org_name) = 'NEGATIVE' THEN 'Negative'
                   WHEN LOWER(hosp.comments) LIKE '%negative%' THEN 'Negative'
                   ELSE 'Not Tested'
               END
           ELSE 'Not Tested'
       END AS cdiff_result
FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents` hosp
WHERE hosp.test_itemid IN (90212, 90221, 90233, 90253)
"""
query_job = client.query(cdiff_query)
df_cdiff = query_job.to_dataframe()

In [ ]:
print(df_ppi.columns, df_abx.columns, df_cdiff.columns)

Index(['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'pharmacy_id',
       'enter_provider_id', 'charttime', 'medication', 'event_txt',
       'scheduletime', 'storetime'],
      dtype='object') Index(['subject_id', 'hadm_id', 'stay_id', 'antibiotic', 'route', 'starttime',
       'stoptime'],
      dtype='object') Index(['subject_id', 'hadm_id', 'chartdate', 'charttime', 'test_itemid',
       'test_name', 'org_name', 'comments', 'cdiff_result'],
      dtype='object')


In [ ]:
df_ppi.columns

Index(['subject_id_cdiff', 'hadm_id_cdiff', 'chartdate_cdiff_cdiff',
       'charttime_cdiff_cdiff', 'test_itemid_cdiff_cdiff',
       'test_name_cdiff_cdiff', 'org_name_cdiff_cdiff', 'comments_cdiff_cdiff',
       'cdiff_result_cdiff_cdiff'],
      dtype='object')

In [ ]:
# add suffix to all columns except subject_id, hadm_id
df_ppi = df_ppi.add_suffix("_ppi")
df_abx = df_abx.add_suffix("_abx")
df_cdiff = df_cdiff.add_suffix("_cdiff")

# rename column name
df_ppi = df_ppi.rename(columns={'subject_id_ppi': 'subject_id'})
df_ppi = df_ppi.rename(columns={'hadm_id_ppi': 'hadm_id'})

df_abx = df_abx.rename(columns={'subject_id_abx': 'subject_id'})
df_abx = df_abx.rename(columns={'hadm_id_abx': 'hadm_id'})

df_cdiff = df_cdiff.rename(columns={'subject_id_cdiff': 'subject_id'})
df_cdiff = df_cdiff.rename(columns={'hadm_id_cdiff': 'hadm_id'})

merge_df = pd.merge(df_ppi, df_abx,how="left",on=["subject_id","hadm_id"])
merge_df = pd.merge(merge_df, df_cdiff,how="left",on=["subject_id","hadm_id"])


In [ ]:
merge_df.columns

Index(['subject_id', 'hadm_id', 'emar_id_ppi', 'emar_seq_ppi', 'poe_id_ppi',
       'pharmacy_id_ppi', 'enter_provider_id_ppi', 'charttime_ppi',
       'medication_ppi', 'event_txt_ppi', 'scheduletime_ppi', 'storetime_ppi',
       'stay_id_abx', 'antibiotic_abx', 'route_abx', 'starttime_abx',
       'stoptime_abx', 'chartdate_cdiff', 'charttime_cdiff',
       'test_itemid_cdiff', 'test_name_cdiff', 'org_name_cdiff',
       'comments_cdiff', 'cdiff_result_cdiff'],
      dtype='object')

In [ ]:
# Convert relevant columns to datetime objects
merge_df['charttime_ppi'] = pd.to_datetime(merge_df['charttime_ppi'])
merge_df['starttime_abx'] = pd.to_datetime(merge_df['starttime_abx'])
merge_df['charttime_cdiff'] = pd.to_datetime(merge_df['charttime_cdiff'])

In [ ]:
merge_df.head()

,subject_id,hadm_id,emar_id_ppi,emar_seq_ppi,poe_id_ppi,pharmacy_id_ppi,enter_provider_id_ppi,charttime_ppi,medication_ppi,event_txt_ppi,...,route_abx,starttime_abx,stoptime_abx,chartdate_cdiff,charttime_cdiff,test_itemid_cdiff,test_name_cdiff,org_name_cdiff,comments_cdiff,cdiff_result_cdiff
0,16679562,20001395,16679562-370,370,16679562-1139,39145761,None,2180-04-24 09:06:00,Omeprazole,Administered,...,NaN,NaT,NaT,NaT,NaT,<NA>,NaN,NaN,NaN,NaN
1,16679562,20001395,16679562-393,393,16679562-1183,39145761,None,2180-04-25 08:12:00,Omeprazole,Administered,...,NaN,NaT,NaT,NaT,NaT,<NA>,NaN,NaN,NaN,NaN
2,16679562,20001395,16679562-414,414,16679562-1183,39145761,None,2180-04-26 08:14:00,Omeprazole,Administered,...,NaN,NaT,NaT,NaT,NaT,<NA>,NaN,NaN,NaN,NaN
3,16679562,20001395,16679562-439,439,16679562-1183,39145761,None,2180-04-27 09:13:00,Omeprazole,Administered,...,NaN,NaT,NaT,NaT,NaT,<NA>,NaN,NaN,NaN,NaN
4,16679562,20001395,16679562-464,464,16679562-1183,39145761,None,2180-04-28 08:01:00,Omeprazole,Administered,...,NaN,NaT,NaT,NaT,NaT,<NA>,NaN,NaN,NaN,NaN


In [ ]:
#Filter for the specified conditions
filtered_df = merge_df[
    merge_df['charttime_ppi'] <= merge_df['charttime_cdiff']
]
print(filtered_df.head())

     subject_id   hadm_id    emar_id_ppi  emar_seq_ppi     poe_id_ppi  \
880    11389314  20011281  11389314-1332          1332  11389314-2831   
881    11389314  20011281  11389314-1358          1358  11389314-2831   
928    19650702  20013839  19650702-1495          1495   19650702-828   
929    19650702  20013839  19650702-1495          1495   19650702-828   
930    19650702  20013839  19650702-1495          1495   19650702-828   

     pharmacy_id_ppi enter_provider_id_ppi       charttime_ppi medication_ppi  \
880         53164087                  None 2190-01-23 14:02:00     Omeprazole   
881         53164087                P12B2C 2190-01-24 08:01:00     Omeprazole   
928         63056655                  None 2118-07-07 09:09:00     Omeprazole   
929         63056655                  None 2118-07-07 09:09:00     Omeprazole   
930         63056655                  None 2118-07-07 09:09:00     Omeprazole   

    event_txt_ppi  ... route_abx       starttime_abx        stoptime_abx  

In [ ]:
# prompt: make a variable = 1 if  c_diff  = "Positive" and ppi_date <= cdiff_date
# else 0

# Create the new column based on the specified conditions
filtered_df['cdiff_case'] = 0  # Initialize the column with 0
filtered_df.loc[(filtered_df['cdiff_result_cdiff'] == 'Positive') & (filtered_df['charttime_ppi'] <= filtered_df['charttime_cdiff']), 'cdiff_case'] = 1


<ipython-input-63-fd01b1bc3071>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['cdiff_case'] = 0  # Initialize the column with 0


In [ ]:
# prompt: group by hadm_id and then get the max of the cdiff_case, retain most of the columns and just aggregate for the cdiff_case column

# Group by 'hadm_id' and get the max of 'cdiff_case', retaining other columns
result_df = filtered_df.groupby('hadm_id').agg(
    {'cdiff_case': 'max'}
).reset_index()

# Merge the aggregated 'cdiff_case' back to the original DataFrame
final_df = pd.merge(filtered_df, result_df, on='hadm_id', suffixes=('', '_agg'))

# Display the resulting DataFrame
final_df.head()


,subject_id,hadm_id,emar_id_ppi,emar_seq_ppi,poe_id_ppi,pharmacy_id_ppi,enter_provider_id_ppi,charttime_ppi,medication_ppi,event_txt_ppi,...,chartdate_cdiff,charttime_cdiff,test_itemid_cdiff,test_name_cdiff,org_name_cdiff,comments_cdiff,cdiff_result_cdiff,cdiff_case,new_column,cdiff_case_agg
0,11389314,20011281,11389314-1332,1332,11389314-2831,53164087,None,2190-01-23 14:02:00,Omeprazole,Administered,...,2190-01-24,2190-01-24 15:59:00,90221,C. difficile PCR,CLOSTRIDIUM DIFFICILE,___,Positive,1,1.0,1
1,11389314,20011281,11389314-1358,1358,11389314-2831,53164087,P12B2C,2190-01-24 08:01:00,Omeprazole,Administered,...,2190-01-24,2190-01-24 15:59:00,90221,C. difficile PCR,CLOSTRIDIUM DIFFICILE,___,Positive,1,1.0,1
2,19650702,20013839,19650702-1495,1495,19650702-828,63056655,None,2118-07-07 09:09:00,Omeprazole,Administered,...,2118-07-08,2118-07-08 10:55:00,90221,C. difficile PCR,None,Negative for toxigenic C. difficile by the Ill...,Negative,0,NaN,0
3,19650702,20013839,19650702-1495,1495,19650702-828,63056655,None,2118-07-07 09:09:00,Omeprazole,Administered,...,2118-07-08,2118-07-08 10:55:00,90221,C. difficile PCR,None,Negative for toxigenic C. difficile by the Ill...,Negative,0,NaN,0
4,19650702,20013839,19650702-1495,1495,19650702-828,63056655,None,2118-07-07 09:09:00,Omeprazole,Administered,...,2118-07-08,2118-07-08 10:55:00,90221,C. difficile PCR,None,Negative for toxigenic C. difficile by the Ill...,Negative,0,NaN,0


In [ ]:
# prompt: summarize final_df cdiff_case

# Assuming final_df is already created as in your provided code.

print(final_df['cdiff_case'].describe())
print(final_df['cdiff_case'].value_counts())


count    977748.000000
mean          0.037755
std           0.190604
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: cdiff_case, dtype: float64
cdiff_case
0    940833
1     36915
Name: count, dtype: int64


In [ ]:
# prompt: can you combine all the same hadm_ids to redo the last step?

# Assuming final_df is already created as in your provided code.

# Group by 'hadm_id' and get the max of 'cdiff_case', retaining other columns
result_df = final_df.groupby('hadm_id').agg(
    {'cdiff_case': 'max'}
).reset_index()

print(result_df['cdiff_case'].describe())
print(result_df['cdiff_case'].value_counts())


count    2073.000000
mean        0.100820
std         0.301163
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: cdiff_case, dtype: float64
cdiff_case
0    1864
1     209
Name: count, dtype: int64


In [ ]:
merge_df['subject_id'].nunique()
merge_df.columns
# count by hadm_id

# antibiotic_date <= cdiff_date

# with cdiff
# ppi_date <= cdiff_date -- case when to define a variable
# antibiotic_date <= cdiff_date

# exclude negative cdiffs for hadm's if positive cdiff is present at least once

# with negative cdiff
# ppi_date
# antibiotic_date <= cdiff_date

# x columns are the ppi, y columns are the abx, z columns are the cdiff
# cdiff_result


Index(['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'pharmacy_id',
       'enter_provider_id', 'charttime_x', 'medication', 'event_txt',
       'scheduletime', 'storetime', 'stay_id', 'antibiotic', 'route',
       'starttime', 'stoptime', 'chartdate', 'charttime_y', 'test_itemid',
       'test_name', 'org_name', 'comments', 'cdiff_result'],
      dtype='object')

In [ ]:
# prompt: make a variable for abx_case = 1 if starttime_abx is not NaT and else 0

# Create the new column 'abx_case'
final_df['abx_case'] = 0
final_df.loc[final_df['starttime_abx'].notna(), 'abx_case'] = 1


In [ ]:
# prompt: summarize abx_case column

# Assuming final_df is already created as in your provided code.

print(final_df['abx_case'].describe())
print(final_df['abx_case'].value_counts())


count    977748.000000
mean          0.994264
std           0.075516
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: abx_case, dtype: float64
abx_case
1    972140
0      5608
Name: count, dtype: int64


In [ ]:
# prompt: can you combine all the same hadm_ids to redo the last step?

# Assuming final_df is already created as in your provided code.

# Group by 'hadm_id' and get the max of 'cdiff_case' and 'abx_case', retaining other columns
result_df = final_df.groupby('hadm_id').agg(
    {'cdiff_case': 'max', 'abx_case': 'max'}
).reset_index()

print(result_df['cdiff_case'].describe())
print(result_df['cdiff_case'].value_counts())

print(result_df['abx_case'].describe())
print(result_df['abx_case'].value_counts())


count    2073.000000
mean        0.100820
std         0.301163
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: cdiff_case, dtype: float64
cdiff_case
0    1864
1     209
Name: count, dtype: int64
count    2073.000000
mean        0.942113
std         0.233586
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: abx_case, dtype: float64
abx_case
1    1953
0     120
Name: count, dtype: int64


In [ ]:
result_df.head()

,hadm_id,cdiff_case,abx_case
0,20011281,1,0
1,20013839,0,1
2,20014219,0,1
3,20015802,0,1
4,20034788,0,1


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(result_df['cdiff_case'], result_df['abx_case'])
print(contingency_table)
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Display the results
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

abx_case      0     1
cdiff_case           
0           106  1758
1            14   195
Chi-square statistic: 0.19167495862392872
p-value: 0.6615262049876169
Degrees of freedom: 1
Expected frequencies:
[[ 107.9015919 1756.0984081]
 [  12.0984081  196.9015919]]
